## Build Training set for doc types

In [2]:
import TWB
import glob

In [3]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
#from nltk.stem import LancasterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
porter = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sparaschiakos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
english_document_data = pd.read_pickle('data/english_document_data.pkl')
stems = pd.read_pickle('data/stemmed_data.pkl')
freq_content = pd.read_pickle('data/freq_content.pkl')

In [48]:
stems.shape

(34525, 2)

In [4]:
# # defining function to check buzz word 
# def exist_(temp,j): 
#     temp2=temp.lower()
#     temp3=temp2.find(j)
#     return temp3>0
    
# dictionary2=['certificate','instruction','poster','protocol', 'presentation',
#              'response','letter','report','map','manual','banner']

# for j in dictionary2:
#     jj=english_document_data.Filename.apply(lambda x: exist_(x,j))
#     rr=pd.DataFrame(columns=[str(j)])
#     rr[str(j)]=jj
#     english_document_data=pd.concat([english_document_data,rr ], axis=1)

In [5]:
# keep only the english ones
stems_en = stems.iloc[english_document_data.index]
freq_content_en = freq_content.iloc[english_document_data.index]

In [6]:
# drop dublicates and create the corpus
stems_en_noDbl = stems_en.drop_duplicates(subset='Content')
len(stems_en_noDbl.index)

6730

In [277]:
# drop duplicates of english_document_data based on content 
english_document_data_noDbl = english_document_data.loc[stems_en_noDbl.index]

In [278]:
# keep df with true in labels
df = english_document_data_noDbl.loc[
    (english_document_data_noDbl['certificate']==True) |
    (english_document_data_noDbl['instruction']==True) |
    (english_document_data_noDbl['poster']==True) |
    (english_document_data_noDbl['protocol']==True) |
    (english_document_data_noDbl['presentation']==True) |
    (english_document_data_noDbl['response']==True) |
    (english_document_data_noDbl['letter']==True) |
    (english_document_data_noDbl['report']==True) |
    (english_document_data_noDbl['map']==True) |
    (english_document_data_noDbl['manual']==True) |
    (english_document_data_noDbl['banner']==True)]

In [279]:
labeled_docs = pd.DataFrame(index= df.index, columns=['document', 'label'])

In [280]:
# keep docs with label
for i in df.index:
    go = False
    for j in df.columns:
        if (df[j][i] == True) & (go == False):
            labeled_docs.document[i] = df.Filename[i]
            labeled_docs.label[i] = j
            go = True

In [281]:
labeled_docs

,document,label
205,English_Volunteer_Newsletter_January_2017.docx,letter
461,201702_Letter_from_the_NGO_Committee_on_Migrat...,letter
692,Labels_for_Emergency_Response_Map.docx,response
1064,CP_protocol_FINAL_with_LSTMH_approval.docx,protocol
1191,2a_Data_and_information_for_epidemic_response_...,response
1193,2b_Laboratory_networks_to_support_outbreak_pre...,response
1195,Publisher_report_2016_final.docx,report
1202,CAFE_newsletter_Apr_2017_for_translation_1.docx,letter
1206,CAFE_newsletter_Apr_2017_for_translation_2.docx,letter
1419,BRONZE_Annual_Report_Template.docx,report


## Compute the relative freqs per word in the corpus per document

In [ ]:
# create the list with all the contents
texts =[]
texts = list(stems_en_noDbl.Content)
token_freq = [ TWB.common.freq(t.split(' ')) for t in texts ]
token_freq[0]

In [59]:
dictionary = list(set(' '.join(texts).split(' ')))
dict_idx = { w : i for (i,w) in enumerate(dictionary) }
# dict_idx

In [60]:
import numpy as np
M = np.zeros((len(texts),len(dictionary)))

In [63]:
for i, text in enumerate(texts):
    M[i,[dict_idx[w] for w in token_freq[i].keys()]] = np.array(list(token_freq[i].values()))/sum(token_freq[i].values())

In [197]:
# the relative freqs per word per document
labeled_set= pd.DataFrame(M, columns=dictionary,index=stems_en_noDbl.index)

In [161]:
labeled_set

,,prosynergi,hangout,sodi,sindhuli,aprenden,sgsa,mainstream,aligncent,ሊረዱህ,...,element,straight,2cqpwhz,ressourc,5312018,ማስፈቀድ,وتمثل,nationalarch,5203070012,brown
0,0.005747,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
7,0.005155,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
13,0.005650,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
16,0.007042,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
19,0.000034,0.0,0.0,0.0,0.0,0.0,0.0,0.000271,0.0,0.0,...,0.000440,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
22,0.000114,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000114,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
29,0.001580,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
30,0.000530,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
106,0.000223,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
107,0.002481,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.002481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [67]:
# keep cols with only digits
digit_stems = []
for col in labeled_set.columns:
    if col.isdigit():
        digit_stems.append(col)  
len(digit_stems)

In [199]:
# compute and keep the amount of digit stems per doc
labeled_set['__digits'] = labeled_set[digit_stems].apply(sum,axis=1)

In [200]:
# drop digit columns
labeled_set = labeled_set.drop(columns=digit_stems)

In [282]:
# create a column for the labels for all the docs and impute as unknown
labeled_set["label"] = 'Unknown'

In [283]:
# impute the knwon labels
for i in labeled_docs.index:
        labeled_set['label'].loc[i] = labeled_docs["label"].loc[i]

In [284]:
labeled_docs['label'].unique()

array(['letter', 'response', 'protocol', 'report', 'presentation', 'map',
       'certificate', 'poster', 'manual', 'instruction'], dtype=object)

In [285]:
# merge instructions and manual
labeled_docs['label'].loc[(labeled_docs['label']=='manual') | 
                          (labeled_docs['label']=='instruction')] = 'manual/instruction'

In [286]:
# save the labeled set of docs 
labeled_set.to_pickle('data/labeled_set_type.pkl')

## PCA

In [5]:
from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd

In [7]:
# load labeled_set
labeled_set = pd.read_pickle('data/labeled_set_type.pkl')

In [8]:
# keep only features
features_doc_types = labeled_set.drop(columns=['label'])

In [9]:
# standarize
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(features_doc_types)
# Apply transform to both the training set and the test set.
train_img = scaler.transform(features_doc_types)

In [10]:
from sklearn.decomposition import PCA
pca = PCA(.95)

In [11]:
pca.fit(train_img)

PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [ ]:
import joblib
print(pca.n_components_)

joblib.dump(pca,'docClassif/pca_type.pkl')

## Training Classifiers

In [192]:
from pandas.tools.plotting import parallel_coordinates
import pandas as pd
from sklearn import svm

from sklearn.metrics import jaccard_similarity_score

#from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_state = 100
#import xgboost as xgb

import pickle
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from sklearn.model_selection import cross_val_score

In [193]:
pca.n_components_

3685

In [288]:
# keep all the docs with label (drop unknown)
features_to_class = features_doc_types.loc[labeled_docs.index]
test_to_class = labeled_set.label.loc[labeled_docs.index]

In [289]:
features_to_class

,,prosynergi,hangout,sodi,sindhuli,aprenden,sgsa,mainstream,aligncent,ሊረዱህ,...,eﬀort,element,straight,2cqpwhz,ressourc,ማስፈቀድ,وتمثل,nationalarch,brown,__digits
205,0.000637,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.021656
461,0.003788,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.003788
692,0.024390,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.195122
1064,0.000267,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000267,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.043977
1191,0.001585,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.012678
1193,0.001181,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.020071
1195,0.000389,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.220062
1202,0.001479,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.026627
1206,0.002381,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.011905
1419,0.001748,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.027972


In [290]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(features_to_class, test_to_class, test_size=0.3)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)


(397, 107251) (397,)
(171, 107251) (171,)


In [298]:
type(X_train)

numpy.ndarray

In [292]:
# transform features from PCA
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [299]:
pd.DataFrame(X_train).to_pickle('X_train_docType.pkl')
pd.DataFrame(X_test).to_pickle('X_test_docType.pkl')

In [ ]:
pd.DataFrame(Y_train).to_pickle('Y_train_docType.pkl')
pd.DataFrame(Y_test).to_pickle('Y_test_docType.pkl')

### Random Forest

In [209]:
# Hyperparameter Optimization for random forest
from scipy.stats import randint as sp_randint

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, make_scorer
from time import time
# initialize the classifier
clf_RF = RandomForestClassifier(n_estimators=10, n_jobs=-1, random_state=100)
# find the number of features for parameter optimization
num_features = train_img.shape[1]

# make scorer#
#auc_scorer = make_scorer(roc_auc_score, needs_threshold=True)#, needs_proba=True

# params
m_depths = [None] + list(range(2, 21, 3))
# specify parameters and distributions to sample from
param_dist_rf = {"max_depth": m_depths, # [None, 2, 5, 8, 11, 14, 17, 20]
                 "max_features": sp_randint(1, num_features),
                 "min_samples_split": sp_randint(2, 100),
                 "min_samples_leaf": sp_randint(1, 100),
                 "bootstrap": [True, False],
                 "criterion": ["gini", "entropy"]}

n_iter_search = 30 # was 20

# run randomized search
random_search_RF = RandomizedSearchCV(clf_RF, 
                                      param_distributions=param_dist_rf,
                                      n_iter=n_iter_search,
                                      random_state=random_state,
                                      n_jobs=-1,
                                      return_train_score=True)

start = time()
random_search_RF.fit(X_train, Y_train)
print("RandomizedSearchCV took {:.2f} seconds for {} candidates"
      " parameter settings.\n".format((time() - start), n_iter_search))

# display the dataframe without the parameters column for better rendering
display(pd.DataFrame(random_search_RF.cv_results_).drop('params', axis=1))

best_parameters_RF = pd.Series(random_search_RF.best_params_)

# display the best set of parameters
display(best_parameters_RF)

RandomizedSearchCV took 1405.56 seconds for 30 candidates parameter settings.



,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_bootstrap,param_criterion,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,...,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,31.223052,5.670263,0.919125,0.919127,True,gini,8,80,49,12,...,0.916825,0.920281,0.918523,0.919427,0.922045,0.917673,4.987006,1.819144,0.002173,0.001085
1,13.210189,3.622221,0.919125,0.919127,True,gini,5,54,67,16,...,0.916825,0.920281,0.918523,0.919427,0.922045,0.917673,7.348636,2.263728,0.002173,0.001085
2,15.783922,3.268891,0.919125,0.919127,True,entropy,None,281,16,62,...,0.916825,0.920281,0.918523,0.919427,0.922045,0.917673,3.045729,0.495901,0.002173,0.001085
3,9.070457,2.349524,0.919125,0.919127,True,gini,8,1674,94,88,...,0.916825,0.920281,0.918523,0.919427,0.922045,0.917673,2.353609,0.594629,0.002173,0.001085
4,12.246951,2.491501,0.919125,0.919127,True,entropy,11,1925,32,3,...,0.916825,0.920281,0.918523,0.919427,0.922045,0.917673,2.476356,1.524231,0.002173,0.001085
5,7.845189,1.305541,0.918913,0.920717,False,entropy,8,613,5,93,...,0.916825,0.920281,0.918523,0.921019,0.921406,0.920852,1.903682,0.220958,0.001890,0.000316
6,13.609464,3.354864,0.919125,0.919127,False,entropy,20,2098,48,67,...,0.916825,0.920281,0.918523,0.919427,0.922045,0.917673,3.125547,0.631350,0.002173,0.001085
7,15.612455,3.010050,0.919125,0.919127,False,gini,20,3016,81,4,...,0.916825,0.920281,0.918523,0.919427,0.922045,0.917673,4.388372,1.317899,0.002173,0.001085
8,11.108730,2.359816,0.919125,0.919127,False,entropy,17,1428,99,65,...,0.916825,0.920281,0.918523,0.919427,0.922045,0.917673,3.069103,0.725675,0.002173,0.001085
9,15.249608,3.295619,0.919125,0.919127,False,entropy,None,3311,31,50,...,0.916825,0.920281,0.918523,0.919427,0.922045,0.917673,1.413587,0.979940,0.002173,0.001085


bootstrap            True
criterion            gini
max_depth               8
max_features           80
min_samples_leaf       49
min_samples_split      12
dtype: object

In [293]:
# Train optimized random forest
model7 = RandomForestClassifier(n_estimators=10 ,
                                min_samples_split= 12,
                                min_samples_leaf=49,
                                bootstrap = True,
                                criterion = 'gini',
                                max_depth = 8,
                                max_features = 80,
                                n_jobs=-1,
                                random_state=100)

model7.fit(X_train, Y_train)
print('train accuracy: '+str(jaccard_similarity_score(model7.predict(X_train), Y_train)))
print('test accuracy: '+str(jaccard_similarity_score(model7.predict(X_test), Y_test)))


train accuracy: 0.5340050377833753
test accuracy: 0.5730994152046783


In [266]:
X_train

,,prosynergi,hangout,sodi,sindhuli,aprenden,sgsa,mainstream,aligncent,ሊረዱህ,...,eﬀort,element,straight,2cqpwhz,ressourc,ማስፈቀድ,وتمثل,nationalarch,brown,__digits
6466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6477,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33775,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [294]:
from sklearn.metrics import confusion_matrix
# Compute confusion matrix train
y_true = Y_train
y_pred = model7.predict(X_train)
cm = confusion_matrix(y_true, y_pred)
print(cm)
# Compute confusion matrix train
y_true = Y_test
y_pred = model7.predict(X_test)
cm = confusion_matrix(y_true, y_pred)
print(cm)

[[  0   0   2   0   0   0   0   0   7   0]
 [  0   0   2   0   0   0   0   0   3   0]
 [  0   0  41   0   0   0   0   0  19   0]
 [  0   0   6   0   0   0   0   0  23   0]
 [  0   0   2   0   0   0   0   0  14   0]
 [  0   0   5   0   0   0   0   0  17   0]
 [  0   0   2   0   0   0   2   0  35   0]
 [  0   0   0   0   0   0   1   0  10   0]
 [  0   0   5   0   0   0   0   0 169   0]
 [  0   0   2   0   0   0   0   0  30   0]]
[[ 0  0  0  0  0  0  0  0  2  0]
 [ 0  0  0  0  0  0  0  0  3  0]
 [ 0  0 21  0  0  0  0  0 14  0]
 [ 0  0  0  0  0  0  1  0  8  0]
 [ 0  0  1  0  0  0  0  0  6  0]
 [ 0  0  1  0  0  0  0  0  4  0]
 [ 0  0  1  0  0  0  1  0 12  0]
 [ 0  0  0  0  0  0  0  0  6  0]
 [ 0  0  3  0  0  0  0  0 76  0]
 [ 0  0  1  0  0  0  0  0 10  0]]


In [296]:
count = Y_test.value_counts() 
count

report          79
letter          35
presentation    14
response        11
manual           9
map              7
protocol         6
poster           5
instruction      3
certificate      2
Name: label, dtype: int64